# Functions

In [ ]:
def createDTM(row, laz_folder, dtm_folder, params, overwrite=False):
    infile = row.originFile
    fname = os.path.basename(infile)
    #suffix = f"Sc{params['scalar']}Sl{params['slope']}Th{params['threshold']}WS{params['win_size']}"
    suffix = os.path.basename(dtm_folder)
    
    #outlaz = os.path.join(laz_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.laz"))
    outtiff = os.path.join(dtm_folder, fname.replace(".laz", f"_{row.subIndex}_{suffix}.tif"))
    
    if os.path.exists(outtiff) and overwrite == False:
        return
    
    filterSMRF = """
    {
        "type": "filters.smrf",
        "returns":"first,last,intermediate,only",
        "cell":%s,
        "scalar": %s,
        "slope": %s,
        "threshold": %s,
        "window": %s     
    }"""
    
    geom = row.geometry.buffer(params['win_size'])
    xmin,ymin,xmax,ymax = geom.bounds
    
    #if os.path.exists(outlaz) and not overwrite:
    #    return
        #pipeline_json = """ "%s" """ % outlaz
    if not os.path.exists(outtiff):
        pipeline_json = """ "%s" """ % infile
        pipeline_json += ',' + filterCrop % (xmin,xmax,ymin,ymax)
        pipeline_json += ',' + filterSMRF % (params['resolution'],params['scalar'], params['slope'], params['threshold'], params['win_size'])
        #pipeline_json += ',' + filterPMF % (8, 0.15, 40, 50, 1.5)
        pipeline_json += ',' + filterGround
        #pipeline_json += ',' + writeLAZ % (outlaz)
        pipeline_json += ',' + writeTIFF % (outtiff, params['resolution'], 0)
    else:
        return outtiff
        
        
    pipeline_json = json_pipe_base % pipeline_json.replace("\\","/")
    
    pipeline = pdal.Pipeline(pipeline_json)
    pipeline.loglevel = 8

    start = datetime.now()
    try:
        pipeline.execute()
    except:
        print(f"Failed for {outtiff}")
    end = datetime.now()

    print(f"{datetime.now()}\t-\tFinished with {os.path.basename(outtiff)} - {end-start} elapsed")
    
    return outtiff